In [1]:
import os
import requests
import zipfile
import io
import pickle
import numpy as np
import pandas as pd
from rdkit.Chem import AllChem
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit import Chem
from rdkit.Chem import AllChem
import molvs

def check_molecule(smiles):
    try:
        mol = molvs.Standardizer().standardize(Chem.MolFromSmiles(smiles))
        if mol is None:
            return None
            
        else:
            if mol.GetNumAtoms() <= 1:
                
                print(f'Error: {smiles} is invalid')
                return None
            else:
                return Chem.MolToSmiles(mol, isomericSmiles=True)
    except:
        print(f'Error: {smiles} is invalid')
        return None
    
# set seed
seed = 0
np.random.seed(seed)

In [ ]:
df = pd.read_csv('/scratch/yg3191/tox24_challenge_submission_yingkai_lab/data/TTR_extra.csv')
del df['Unnamed: 0']
df['Source'] = 'CPI2M'
df.to_csv('/scratch/yg3191/tox24_challenge_submission_yingkai_lab/data/TTR_extra.csv', index=False)

,Unnamed: 0,smiles,y
0,388,COc1ccc(/C=C/C(=O)OCCc2ccccc2)cc1O,23.750
1,389,COc1ccc(/C=C/C(=O)OCCc2ccccc2)cc1OC,12.750
2,390,O=C(/C=C/c1ccc(O)c(O)c1)OCc1ccccc1,21.000
3,391,Cc1cc(/C=C/c2cccc(C(=O)O)c2)cc(C)c1O,30.625
4,392,Cc1c(C(=O)O)coc1/C=C/c1ccc(Cl)o1,33.375
...,...,...,...
210,598,O=C1Nc2ccc(I)cc2C1=Nc1ccccc1,14.125
211,599,O=C1Nc2ccc([N+](=O)[O-])cc2C1=Nc1ccc(I)cc1,14.125
212,600,O=C1Nc2ccccc2C1=Nc1ccccc1,14.125
213,601,O=Cc1ccc(Oc2ccc([N+](=O)[O-])cc2[N+](=O)[O-])c...,13.575


In [13]:
with open('5_fold_5_split.pl', 'rb') as f:
    data = pickle.load(f)
ref_data = pd.read_csv('test_data.csv')
uni_seq_dict = ref_data.set_index('Uniprot_id')['Sequence'].to_dict()
test_data = ref_data[ref_data['split'] == 'test']
for idx, d in enumerate(data):
    os.makedirs(f'repeat_{idx}', exist_ok=True)
    for fold in d.keys():
        fold_train = d[fold]['train']
        fold_train['smiles'] = fold_train['cSMILES']
        fold_train['y'] = fold_train['activity']
        fold_train['exp_mean [nM]'] = None
        fold_train['cliff_mol'] = 0
        fold_train['split'] = 'train'
        fold_train['Uniprot_id'] = 'P02766'
        fold_train['Sequence'] = uni_seq_dict['P02766']
        fold_train['smiles'] = fold_train['smiles'].map(check_molecule)
        fold_train = fold_train.dropna(subset=['smiles'])
        
        fold_val = d[fold]['valid'] 
        fold_val['smiles'] = fold_val['cSMILES']
        fold_val['y'] = fold_val['activity']
        fold_val['exp_mean [nM]'] = None
        fold_val['cliff_mol'] = 0
        fold_val['split'] = 'valid'
        fold_val['Uniprot_id'] = 'P02766'
        fold_val['Sequence'] = uni_seq_dict['P02766']
        fold_val['smiles'] = fold_val['smiles'].map(check_molecule)
        fold_val = fold_val.dropna(subset=['smiles'])

        fold_data = pd.concat([fold_train, fold_val], axis=0)       
        fold_data = pd.concat([fold_data, test_data], axis=0).reset_index(drop=True)
        fold_data.to_csv(f'repeat_{idx}/fold_{fold}.csv', index=False)
    
fold_data

,SMILES,activity,cSMILES,smiles,y,exp_mean [nM],cliff_mol,split,Uniprot_id,Sequence,Activity_Type,Note
0,CC1(C)[C@@H]2C[C@H]1C(=C)CC2,12.3,C=C1CC[C@H]2C[C@@H]1C2(C)C,C=C1CC[C@H]2C[C@@H]1C2(C)C,12.3,NaN,0,train,P02766,MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,NaN,NaN
1,COC(=O)C(C)OC1=CC=C(OC2=CC=C(Cl)C=C2Cl)C=C1,94.1,COC(=O)C(C)Oc1ccc(Oc2ccc(Cl)cc2Cl)cc1,COC(=O)C(C)Oc1ccc(Oc2ccc(Cl)cc2Cl)cc1,94.1,NaN,0,train,P02766,MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,NaN,NaN
2,CCCCCCCCCCCCCC(=O)OCC(O)CO,38.1,CCCCCCCCCCCCCC(=O)OCC(O)CO,CCCCCCCCCCCCCC(=O)OCC(O)CO,38.1,NaN,0,train,P02766,MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,NaN,NaN
3,CC(=O)O[C@@H]1C[C@@H]2CC[C@@]1(C)C2(C)C,11.8,CC(=O)O[C@@H]1C[C@@H]2CC[C@@]1(C)C2(C)C,CC(=O)O[C@@H]1C[C@@H]2CC[C@@]1(C)C2(C)C,11.8,NaN,0,train,P02766,MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,NaN,NaN
4,CCCCCCCCCC=CCC1CC(=O)OC1=O,98.4,CCCCCCCCCC=CCC1CC(=O)OC1=O,CCCCCCCCCC=CCC1CC(=O)OC1=O,98.4,NaN,0,train,P02766,MASHRLLLLCLAGLVFVSEAGPTGTGESKCPLMVKVLDAVRGSPAI...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1797,NaN,NaN,NaN,COc1ccc(C=O)cc1OC,28.9,NaN,0,test,P02766_3d7p,CPLMVKVLDAVRGSPAINVAVHVFRKAADDTWEPFASGKTSESGEL...,NaN,NaN
1798,NaN,NaN,NaN,C=CC1(C)OC(=O)N(c2cc(Cl)cc(Cl)c2)C1=O,90.1,NaN,0,test,P02766_3d7p,CPLMVKVLDAVRGSPAINVAVHVFRKAADDTWEPFASGKTSESGEL...,NaN,NaN
1799,NaN,NaN,NaN,COP(=O)(OC)O/C(=C\Cl)c1cc(Cl)c(Cl)cc1Cl,100.9,NaN,0,test,P02766_3d7p,CPLMVKVLDAVRGSPAINVAVHVFRKAADDTWEPFASGKTSESGEL...,NaN,NaN
1800,NaN,NaN,NaN,[O-]n1ccccc1=S.[O-]n1ccccc1=S.[ZnH2+2],21.1,NaN,0,test,P02766_3d7p,CPLMVKVLDAVRGSPAINVAVHVFRKAADDTWEPFASGKTSESGEL...,NaN,NaN


In [14]:
# generating molecular fingerprints and descriptors
def Morgan_fingerprint(smiles):
    mols = [Chem.MolFromSmiles(i) for i in smiles]
    fps = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048) for mol in mols]
    return fps

def feature_pipeline(df):
    morgan_fps = Morgan_fingerprint(df['smiles'])
    morgan_fps_name = [f'Morgan_{i}' for i in range(len(morgan_fps[0]))]
    # combine all features and organize them into a dataframe
    df_feat = pd.DataFrame(np.array(morgan_fps), columns=morgan_fps_name)
    print('Morgan fingerprints shape:', df_feat.shape)
    assert len(df_feat) == len(df)
    df_feat['y'] = df['y'].values
    return df_feat

In [15]:
df = pd.read_csv('repeat_0/fold_0.csv')
train, valid, test = df[df['split'] == 'train'], df[df['split'] == 'valid'], df[df['split'] == 'test']
train_data, valid_data, test_data = feature_pipeline(train), feature_pipeline(valid), feature_pipeline(test)

Morgan fingerprints shape: (961, 2048)
Morgan fingerprints shape: (241, 2048)
Morgan fingerprints shape: (600, 2048)


# Model training

In [16]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

def calc_score(label, pred):
    # calculate R2, RMSE, MAE, PCC
    r2 = r2_score(label, pred)
    rmse = np.sqrt(mean_squared_error(label, pred))
    mae = mean_absolute_error(label, pred)
    pcc, _ = pearsonr(label, pred)
    return r2, rmse, mae, pcc

# train a XGBoost model
import xgboost as xgb

df = pd.read_csv('repeat_0/fold_0.csv')
train, valid, test = df[df['split'] == 'train'], df[df['split'] == 'valid'], df[df['split'] == 'test']
train_data, valid_data, test_data = feature_pipeline(train), feature_pipeline(valid), feature_pipeline(test)

dtrain = xgb.DMatrix(train_data.drop(columns=['y']), label=train_data['y'])
dvalid = xgb.DMatrix(valid_data.drop(columns=['y']), label=valid_data['y'])
dtest = xgb.DMatrix(test_data.drop(columns=['y']), label=test_data['y'])
param = {'max_depth': 5, 'eta': 0.3, 'objective': 'reg:squarederror'}
param['nthread'] = 4
param['eval_metric'] = 'rmse'
evallist = [(dvalid, 'eval'), (dtrain, 'train')]
num_round = 100
# mute verbose
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10, verbose_eval=False)
# calculate the RMSE
pred = bst.predict(dtest)
preds = pd.DataFrame({'y': test_data['y'], 'pred': pred})
r2, rmse, mae, pcc = calc_score(preds['y'], preds['pred'])
print(f'R2: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}, PCC: {pcc:.4f}')

Morgan fingerprints shape: (961, 2048)
Morgan fingerprints shape: (241, 2048)
Morgan fingerprints shape: (600, 2048)


c:\Users\Administrator\.conda\envs\cheminfo\lib\site-packages\xgboost\core.py:726: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)


R2: 0.4417, RMSE: 26.7272, MAE: 20.5504, PCC: 0.6700


In [18]:
# Grid search for hyperparameters
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [3, 4, 5, 6, 7],
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'n_estimators': [100, 200, 300, 400, 500]
}

xgb_model = xgb.XGBRegressor()
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(train_data.drop(columns=['y']), train_data['y'])
print(grid_search.best_params_)


Fitting 5 folds for each of 125 candidates, totalling 625 fits
{'learning_rate': 0.2, 'max_depth': 4, 'n_estimators': 100}


In [19]:
result_collect = []
for repeat in range(5):
    for fold in range(5):
        df = pd.read_csv(f'repeat_{repeat}/fold_{fold}.csv')
        train, valid, test = df[df['split'] == 'train'], df[df['split'] == 'valid'], df[df['split'] == 'test']
        train_data, valid_data, test_data = feature_pipeline(train), feature_pipeline(valid), feature_pipeline(test)

        dtrain = xgb.DMatrix(train_data.drop(columns=['y']), label=train_data['y'])
        dvalid = xgb.DMatrix(valid_data.drop(columns=['y']), label=valid_data['y'])
        dtest = xgb.DMatrix(test_data.drop(columns=['y']), label=test_data['y'])
        param = {'max_depth': 7, 'learning_rate': 0.3, 'n_estimators': 500}
        param['eval_metric'] = 'rmse'
        evallist = [(dvalid, 'eval'), (dtrain, 'train')]
        num_round = 100
        # mute verbose
        bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=10, verbose_eval=False)
        # calculate the RMSE, 
        pred = bst.predict(dtest)
        preds = pd.DataFrame({'y': test_data['y'], 'pred': pred})
        preds['smiles'] = test['smiles'].values    
        r2, rmse, mae, pcc = calc_score(preds['y'], preds['pred'])
        result_collect.append([repeat, fold, r2, rmse, mae, pcc])
        # preds.to_csv(f'repeat_{repeat}/fold_{fold}_pred.csv', index=False)
result_mean = np.mean(result_collect, axis=0)
result_std = np.std(result_collect, axis=0)
print(f'R2: {result_mean[2]:.4f}±{result_std[2]:.4f}, RMSE: {result_mean[3]:.4f}±{result_std[3]:.4f}, MAE: {result_mean[4]:.4f}±{result_std[4]:.4f}, PCC: {result_mean[5]:.4f}±{result_std[5]:.4f}')

Morgan fingerprints shape: (961, 2048)
Morgan fingerprints shape: (241, 2048)
Morgan fingerprints shape: (600, 2048)


c:\Users\Administrator\.conda\envs\cheminfo\lib\site-packages\xgboost\core.py:726: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
c:\Users\Administrator\.conda\envs\cheminfo\lib\site-packages\xgboost\core.py:160: UserWarning: [19:11:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Morgan fingerprints shape: (961, 2048)
Morgan fingerprints shape: (241, 2048)
Morgan fingerprints shape: (600, 2048)


c:\Users\Administrator\.conda\envs\cheminfo\lib\site-packages\xgboost\core.py:726: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
c:\Users\Administrator\.conda\envs\cheminfo\lib\site-packages\xgboost\core.py:160: UserWarning: [19:11:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Morgan fingerprints shape: (962, 2048)
Morgan fingerprints shape: (240, 2048)
Morgan fingerprints shape: (600, 2048)


...

Morgan fingerprints shape: (962, 2048)
Morgan fingerprints shape: (240, 2048)
Morgan fingerprints shape: (600, 2048)


c:\Users\Administrator\.conda\envs\cheminfo\lib\site-packages\xgboost\core.py:726: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
c:\Users\Administrator\.conda\envs\cheminfo\lib\site-packages\xgboost\core.py:160: UserWarning: [19:12:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


Morgan fingerprints shape: (962, 2048)
Morgan fingerprints shape: (240, 2048)
Morgan fingerprints shape: (600, 2048)


c:\Users\Administrator\.conda\envs\cheminfo\lib\site-packages\xgboost\core.py:726: FutureWarning: Pass `evals` as keyword args.
  warnings.warn(msg, FutureWarning)
c:\Users\Administrator\.conda\envs\cheminfo\lib\site-packages\xgboost\core.py:160: UserWarning: [19:12:28] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


R2: 0.4526±0.0276, RMSE: 26.4566±0.6628, MAE: 19.9034±0.4725, PCC: 0.6811±0.0170


In [8]:
# calculate the ensemble RMSE
ensemble_preds = []
for repeat in range(5):
    for fold in range(5):
        preds = pd.read_csv(f'repeat_{repeat}/fold_{fold}_pred.csv')
        ensemble_preds.append(preds['pred'].values)
ensemble_preds = np.mean(ensemble_preds, axis=0)
ensemble_preds = pd.DataFrame({'y': preds['y'], 'pred': ensemble_preds})
ensemble_preds['smiles'] = test['smiles'].values
ensemble_rmse = np.sqrt(np.mean((ensemble_preds['pred'] - ensemble_preds['y'])**2))
print(f'Ensemble RMSE: {ensemble_rmse}')

Ensemble RMSE: 24.901475830888188
